In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install transformers evaluate bert-score

In [ ]:
# remove these character patterns from the strings in the data frame: '\n', '###',[' and '] and replace them with an empty string
raw_df = pd.read_csv('/content/drive/Othercomputers/My MacBook Pro/Thesis_Project/Data/translation_test/model_responses.csv')


In [ ]:
raw_df.head()

,Label,length,input,human-expert,gpt-4o,Tiny Llama Unsloth,aya-23-8B,Llama & Assistant,Llama 3 Unsloth,Meta-Llama-3-8B-Instruct,Llama 2 Unsloth,mistral unsloth,mistral-7B-instruct-v0.3,Llama 3 ORPO
0,2,5,Weledele achtbaare heer en heeren!,Right honourable lord and lords!,Most honorable sir and sirs!,['\n(1) Keeping the exact sentence structure'],['Dearest noble lord and lords!\n\n'],['[signed] Honourable lord and lords'],['Honourable lord and lords!'],"['Please translate the sentence ""Weledele acht']",['[signed] Honourable lord and lords'],['Honourable lord and lords!'],['Worthy honorable sirs and lords'],['Please translate the early modern Dutch text...
1,2,10,"In mijn kennis, [get.] J: V: D: Poel, g: klerk.","In my knowledge [signed], J. V. D. Poel, sworn...","In my knowledge, [signed] J: V: D: Poel, g: cl...",['\n### Instruction:\nYou are a helpful assist...,"['In my knowledge, [get.] J: V: D: Poel, g']","['In my knowledge, [signed] J. V. D. Poel, swo...","['In my knowledge, [signed] J. V. D. Poel, swo...","['In my knowledge, [get.] J: V: D: Poel, a cle...","['In my knowledge, [signed] J. V. D. Poel, swo...","['In my knowledge, [signed] J. V. D. Poel, swo...","['In my knowledge, J. V. D. Poel, gentleman, c...","['In my knowledge, [get.] J: V: D: Poel, gentl..."
2,8,5,Waar haar soon is gebleven?,Where her son has been?,Where has her son gone?,"['\nWhere has she been, besides']",['Where is your husband?'],['Where her son stayed?'],['Where her son is left?'],['Where her son is left? \n\n'],['Where her son has stayed?'],['Where her son stayed?'],['Where has she stayed?'],['Where her son is left? \n\n']
3,10,35,"Zoo zoude men al eens verondersteld zijnde, de...",Even if one would presume [that] the ragiadoor...,"So it would have once been supposed, if the ra...",['\n### Instruction:\n\nYou are a helpful assi...,"['So they would have said, if the ragiadoor ha...",['Thus presuming [that] the ragiadoor had said...,"['One would have assumed thus, that the ragiad...",['The English translation of the early modern ...,"['Therefore, if one would grant that the ragia...","['Supposing one were to suppose, the ragiadoor...",['If it were supposed that the rajah had said ...,['If one were to assume that the ragiadoor had...
4,5,11,Antw: Hij heeft geen seeker getal maar wel een...,"Answer: He did not demand a certain number, bu...",Answer: He has not demanded any specific numbe...,['\n(8) Ant Ant Ant Ant Ant Ant Ant Ant Ant An...,['He has no such great number but some few fig...,['Answer: He has not given a certain number bu...,['Answer: He did not give any seeker but did a...,['Please translate the early modern Dutch sent...,['Answer: He has not given a certain number bu...,"['Answer: He did not demand a certain number, ...",['Answer: Does he not have a seeker number but...,['Please translate the given early modern Dutc...


In [ ]:
def remove_sequences(text):
    if isinstance(text, str):
        text = text.replace('[\'', '').replace('\']', '').replace('###', '').replace('\\n', '').replace('Here is the translation of the early modern Dutch text sentence into contemporary English text:\\n\\n', '')
    return text

In [ ]:
raw_df.iloc[:, 5:] = raw_df.iloc[:, 5:].applymap(remove_sequences)


In [ ]:
raw_df.head()

,Label,length,input,human-expert,gpt-4o,Tiny Llama Unsloth,aya-23-8B,Llama & Assistant,Llama 3 Unsloth,Meta-Llama-3-8B-Instruct,Llama 2 Unsloth,mistral unsloth,mistral-7B-instruct-v0.3,Llama 3 ORPO
0,2,5,Weledele achtbaare heer en heeren!,Right honourable lord and lords!,Most honorable sir and sirs!,(1) Keeping the exact sentence structure,Dearest noble lord and lords!,[signed] Honourable lord and lords,Honourable lord and lords!,"Please translate the sentence ""Weledele acht",[signed] Honourable lord and lords,Honourable lord and lords!,Worthy honorable sirs and lords,Please translate the early modern Dutch text s...
1,2,10,"In mijn kennis, [get.] J: V: D: Poel, g: klerk.","In my knowledge [signed], J. V. D. Poel, sworn...","In my knowledge, [signed] J: V: D: Poel, g: cl...",Instruction:You are a helpful assistant desig...,"In my knowledge, [get.] J: V: D: Poel, g","In my knowledge, [signed] J. V. D. Poel, sworn","In my knowledge, [signed] J. V. D. Poel, sworn...","In my knowledge, [get.] J: V: D: Poel, a clerk","In my knowledge, [signed] J. V. D. Poel, sworn","In my knowledge, [signed] J. V. D. Poel, sworn","In my knowledge, J. V. D. Poel, gentleman, clerk.","In my knowledge, [get.] J: V: D: Poel, gentlem..."
2,8,5,Waar haar soon is gebleven?,Where her son has been?,Where has her son gone?,"Where has she been, besides",Where is your husband?,Where her son stayed?,Where her son is left?,Where her son is left?,Where her son has stayed?,Where her son stayed?,Where has she stayed?,Where her son is left?
3,10,35,"Zoo zoude men al eens verondersteld zijnde, de...",Even if one would presume [that] the ragiadoor...,"So it would have once been supposed, if the ra...",Instruction:You are a helpful assistant desig...,"So they would have said, if the ragiadoor had ...",Thus presuming [that] the ragiadoor had said s...,"One would have assumed thus, that the ragiadoo...",The English translation of the early modern Du...,"Therefore, if one would grant that the ragiado...","Supposing one were to suppose, the ragiadoor w...",If it were supposed that the rajah had said su...,If one were to assume that the ragiadoor had s...
4,5,11,Antw: Hij heeft geen seeker getal maar wel een...,"Answer: He did not demand a certain number, bu...",Answer: He has not demanded any specific numbe...,(8) Ant Ant Ant Ant Ant Ant Ant Ant Ant Ant An...,He has no such great number but some few figures?,Answer: He has not given a certain number but ...,Answer: He did not give any seeker but did ask...,Please translate the early modern Dutch senten...,Answer: He has not given a certain number but ...,"Answer: He did not demand a certain number, bu...",Answer: Does he not have a seeker number but h...,Please translate the given early modern Dutch ...


In [ ]:
# Split the data so that the first 3 columns are one dataframe and the rest are another dataframe
df1 = raw_df.iloc[:, :3]
df2 = raw_df.iloc[:, 3:]

In [ ]:
from nltk.translate import meteor
from nltk import word_tokenize
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer
import evaluate

In [ ]:
scorer = BERTScorer(model_type='bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
meteor = evaluate.load('meteor')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# Copy the second datafram to two new dataframes
bert_data = df2.copy()
meteor_data = df2.copy()

In [ ]:
print(df2.iloc[2,0])
df2.head()

Where her son has been?


,human-expert,gpt-4o,Tiny Llama Unsloth,aya-23-8B,Llama & Assistant,Llama 3 Unsloth,Meta-Llama-3-8B-Instruct,Llama 2 Unsloth,mistral unsloth,mistral-7B-instruct-v0.3,Llama 3 ORPO
0,Right honourable lord and lords!,Most honorable sir and sirs!,(1) Keeping the exact sentence structure,Dearest noble lord and lords!,[signed] Honourable lord and lords,Honourable lord and lords!,"Please translate the sentence ""Weledele acht",[signed] Honourable lord and lords,Honourable lord and lords!,Worthy honorable sirs and lords,Please translate the early modern Dutch text s...
1,"In my knowledge [signed], J. V. D. Poel, sworn...","In my knowledge, [signed] J: V: D: Poel, g: cl...",Instruction:You are a helpful assistant desig...,"In my knowledge, [get.] J: V: D: Poel, g","In my knowledge, [signed] J. V. D. Poel, sworn","In my knowledge, [signed] J. V. D. Poel, sworn...","In my knowledge, [get.] J: V: D: Poel, a clerk","In my knowledge, [signed] J. V. D. Poel, sworn","In my knowledge, [signed] J. V. D. Poel, sworn","In my knowledge, J. V. D. Poel, gentleman, clerk.","In my knowledge, [get.] J: V: D: Poel, gentlem..."
2,Where her son has been?,Where has her son gone?,"Where has she been, besides",Where is your husband?,Where her son stayed?,Where her son is left?,Where her son is left?,Where her son has stayed?,Where her son stayed?,Where has she stayed?,Where her son is left?
3,Even if one would presume [that] the ragiadoor...,"So it would have once been supposed, if the ra...",Instruction:You are a helpful assistant desig...,"So they would have said, if the ragiadoor had ...",Thus presuming [that] the ragiadoor had said s...,"One would have assumed thus, that the ragiadoo...",The English translation of the early modern Du...,"Therefore, if one would grant that the ragiado...","Supposing one were to suppose, the ragiadoor w...",If it were supposed that the rajah had said su...,If one were to assume that the ragiadoor had s...
4,"Answer: He did not demand a certain number, bu...",Answer: He has not demanded any specific numbe...,(8) Ant Ant Ant Ant Ant Ant Ant Ant Ant Ant An...,He has no such great number but some few figures?,Answer: He has not given a certain number but ...,Answer: He did not give any seeker but did ask...,Please translate the early modern Dutch senten...,Answer: He has not given a certain number but ...,"Answer: He did not demand a certain number, bu...",Answer: Does he not have a seeker number but h...,Please translate the given early modern Dutch ...


In [ ]:
import ast


In [ ]:
print(len(df2))

230


In [ ]:
# Iterate through the columns of the dataframes and calculate the BERT score and METEOR score for each row
for i in range(1,len(df2.columns)):
  for j in range(0,len(df2)):

    # Calculate the BERT score
    P, R, F1 = scorer.score([df2.iloc[j, i]], [df2.iloc[j, 0]])
    bert_data.iloc[j, i] = F1.item()

    # Calculate the METEOR score
    value = meteor.compute(predictions=[df2.iloc[j, i]], references=[df2.iloc[j, 0]])

    # Convert the string to a dictionary
    dict_value = ast.literal_eval(str(value))

    # Extract the float value
    float_value = dict_value['meteor']

    meteor_data.iloc[j, i] = float_value

In [ ]:
meteor_data.head()


,human-expert,gpt-4o,Tiny Llama Unsloth,aya-23-8B,Llama & Assistant,Llama 3 Unsloth,Meta-Llama-3-8B-Instruct,Llama 2 Unsloth,mistral unsloth,mistral-7B-instruct-v0.3,Llama 3 ORPO
0,Right honourable lord and lords!,0.25,0.0,0.661458,0.650615,0.844068,0.0,0.650615,0.844068,0.433145,0.0
1,"In my knowledge [signed], J. V. D. Poel, sworn...",0.680745,0.0,0.504386,0.865585,0.92864,0.541503,0.865585,0.865585,0.708343,0.541503
2,Where her son has been?,0.62,0.25,0.169492,0.635593,0.625,0.625,0.806667,0.635593,0.254237,0.625
3,Even if one would presume [that] the ragiadoor...,0.634145,0.060533,0.541492,0.604839,0.590993,0.547833,0.687289,0.55198,0.509259,0.523694
4,"Answer: He did not demand a certain number, bu...",0.479167,0.0,0.16129,0.477707,0.485232,0.031646,0.477707,0.777418,0.334611,0.031447


In [ ]:
bert_data.head()

,human-expert,gpt-4o,Tiny Llama Unsloth,aya-23-8B,Llama & Assistant,Llama 3 Unsloth,Meta-Llama-3-8B-Instruct,Llama 2 Unsloth,mistral unsloth,mistral-7B-instruct-v0.3,Llama 3 ORPO
0,Right honourable lord and lords!,0.727323,0.299503,0.771552,0.661703,0.935674,0.325597,0.661703,0.935674,0.645978,0.275675
1,"In my knowledge [signed], J. V. D. Poel, sworn...",0.713819,0.326048,0.639065,0.882132,0.917692,0.689566,0.882132,0.882132,0.842085,0.714579
2,Where her son has been?,0.870288,0.653663,0.671045,0.852473,0.850957,0.850957,0.9476,0.852473,0.726576,0.850957
3,Even if one would presume [that] the ragiadoor...,0.745679,0.351198,0.723704,0.811631,0.762275,0.675229,0.800126,0.7414,0.656428,0.73536
4,"Answer: He did not demand a certain number, bu...",0.719757,0.272717,0.579946,0.641005,0.682957,0.330118,0.641005,0.849707,0.629118,0.32897


In [ ]:
meteor_data = pd.concat([df1, meteor_data], axis=1)
bert_data = pd.concat([df1, bert_data], axis=1)

In [ ]:
meteor_data.head()

,Label,length,input,human-expert,gpt-4o,Tiny Llama Unsloth,aya-23-8B,Llama & Assistant,Llama 3 Unsloth,Meta-Llama-3-8B-Instruct,Llama 2 Unsloth,mistral unsloth,mistral-7B-instruct-v0.3,Llama 3 ORPO
0,2,5,Weledele achtbaare heer en heeren!,Right honourable lord and lords!,0.25,0.0,0.661458,0.650615,0.844068,0.0,0.650615,0.844068,0.433145,0.0
1,2,10,"In mijn kennis, [get.] J: V: D: Poel, g: klerk.","In my knowledge [signed], J. V. D. Poel, sworn...",0.680745,0.0,0.504386,0.865585,0.92864,0.541503,0.865585,0.865585,0.708343,0.541503
2,8,5,Waar haar soon is gebleven?,Where her son has been?,0.62,0.25,0.169492,0.635593,0.625,0.625,0.806667,0.635593,0.254237,0.625
3,10,35,"Zoo zoude men al eens verondersteld zijnde, de...",Even if one would presume [that] the ragiadoor...,0.634145,0.060533,0.541492,0.604839,0.590993,0.547833,0.687289,0.55198,0.509259,0.523694
4,5,11,Antw: Hij heeft geen seeker getal maar wel een...,"Answer: He did not demand a certain number, bu...",0.479167,0.0,0.16129,0.477707,0.485232,0.031646,0.477707,0.777418,0.334611,0.031447


In [ ]:
# Export the dataframes to csv files
bert_data.to_csv('/content/drive/Othercomputers/My MacBook Pro/Thesis_Project/Data/translation_test/bert_scores.csv')
meteor_data.to_csv('/content/drive/Othercomputers/My MacBook Pro/Thesis_Project/Data/translation_test/meteor_scores.csv')